In [1]:
⍝| echo: false
]box on -style=max
]rows on
⎕io←0

done

┌→────────────────┐
│Was ON -style=min│
└─────────────────┘

┌→──────┐
│Was OFF│
└───────┘

# Intro

This is another classic computer science problem. You are given a knapsack and the maximum weight it can carry. You also have a list of items. Each item has a value and a weight associated with it. Which items do you take to maximize the value you can fit into the knapsack?

Why is this hard? The possible solution space is big and grows fast. If you want to check every combination of items and then take the max you have $2^i$ combinations to check. This is unworkable for even small numbers of items.

The approach here is branch and bound with relaxation. See [Discrete Optimization](https://www.coursera.org/learn/discrete-optimization) for a detailed explanation. The idea is to organize the solution space into a tree and ignore branches that we know wont generate good solutions. 



# Data

Lets start with a small knapsack. 4 items, maximum capacity 11. This is easy to do even by hand but once we scale it up not so much.

Item one is \\$8 and weighs 4 pounds, item 2 is \\$10 and weighs 5 pounds.

In [2]:
⎕←item_count←4 
⎕←max_weight←11
⎕←items←(8 4) (10 5) (15 8) (4 3)

4

11

┌→──────────────────────────┐
│ ┌→──┐ ┌→───┐ ┌→───┐ ┌→──┐ │
│ │8 4│ │10 5│ │15 8│ │4 3│ │
│ └~──┘ └~───┘ └~───┘ └~──┘ │
└∊──────────────────────────┘

In [3]:
⎕←ks4←item_count max_weight items

┌→───────────────────────────────────┐
│ ┌→──────────────────────────┐ │
│ 4 11 │ ┌→──┐ ┌→───┐ ┌→───┐ ┌→──┐ │ │
│ │ │8 4│ │10 5│ │15 8│ │4 3│ │ │
│ │ └~──┘ └~───┘ └~───┘ └~──┘ │ │
│ └∊──────────────────────────┘ │
└∊───────────────────────────────────┘

A solution will be a list of 1s or 0s in the same order as our item list. Each 1 or 0 will represent a decision to either take or leave that particular item. 

1 1 0 0 means we take items 1 and 2 and leave items 3 and 4. The value of the knapsack then is the value of item 1 plus the value of item 2 provided they both fit in the knapsack. If the weight exceeds the capacity of the knapsack the value is 0.

# Branch

To organize the solution space into a tree is very simple. You break it into decisions. Do I take item 1 yes or no. Then for each of those decisions you ask do I take item 2 yes or no. 

In [4]:
⎕←⍬
⎕←⍪0 1
⎕←⍪(0 0) (0 1) (1 0) (1 1)

┌⊖┐
│0│
└~┘

┌→┐
↓0│
│1│
└~┘

┌→──────┐
↓ ┌→──┐ │
│ │0 0│ │
│ └~──┘ │
│ ┌→──┐ │
│ │0 1│ │
│ └~──┘ │
│ ┌→──┐ │
│ │1 0│ │
│ └~──┘ │
│ ┌→──┐ │
│ │1 1│ │
│ └~──┘ │
└∊──────┘

We take a partial solution and transform it into two slightly more complete solutions by making two copies and appending a 1 onto the first copy and 0 onto the other.

In [5]:
branch←{⍪,((,/,∘0),(,/,∘1))⍤1⊢⍵}
⎕←branch ⍬
⎕←branch⍣2⊢⍬
⎕←branch⍣3⊢⍬

┌→┐
↓0│
│1│
└~┘

┌→──────┐
↓ ┌→──┐ │
│ │0 0│ │
│ └~──┘ │
│ ┌→──┐ │
│ │0 1│ │
│ └~──┘ │
│ ┌→──┐ │
│ │1 0│ │
│ └~──┘ │
│ ┌→──┐ │
│ │1 1│ │
│ └~──┘ │
└∊──────┘

┌→────────┐
↓ ┌→────┐ │
│ │0 0 0│ │
│ └~────┘ │
│ ┌→────┐ │
│ │0 0 1│ │
│ └~────┘ │
│ ┌→────┐ │
│ │0 1 0│ │
│ └~────┘ │
│ ┌→────┐ │
│ │0 1 1│ │
│ └~────┘ │
│ ┌→────┐ │
│ │1 0 0│ │
│ └~────┘ │
│ ┌→────┐ │
│ │1 0 1│ │
│ └~────┘ │
│ ┌→────┐ │
│ │1 1 0│ │
│ └~────┘ │
│ ┌→────┐ │
│ │1 1 1│ │
│ └~────┘ │
└∊────────┘

# Bound

To prune our tree we need two things. A valid solution we can compare the value to. And a way of generating a value we think is good predictor for how well a branch will perform. Without evaluating the branch we need to figure out how well it will do. And this is where relaxation comes in. To guess how well this will perform we relax the constraints a little to create a much easier problem. We will come up with solutions that are not in the orginal solution space but if we are careful that is ok.

Getting a start solution is easy. We sort the values by item density and take items until the knapsack is full. If an item does not fit we skip it and check the next item.



## Helpers

Lets start off with some helper functions. 
weight and value both take a knapsack and a partial or full solution. Returns the weight or the value of the given solution

In [140]:
]dinput
weight←{
    c mw is←⍺           ⍝ split the right argument into components count, max weight, and items
    vs ws←↓⍉↑is         ⍝ extract a list of values and a list of weights from the item lis
    psolution←c↑⍵       ⍝ pad the partial solution with 0s to the lenght of the item list
    +/ws∧psolution      ⍝ use the partial solution as a mask to select the weights of the items we take and sum them
}

In [10]:
]dinput
value←{
    c mw is←⍺
    mw≤⍺ weight ⍵ :0 ⍝ if the weight is more than the capacity the value is 0
    vs ws←↓⍉↑is
    psolution←c↑⍵
    +/vs∧psolution
}

## Greedy
Looks at the items in order. If it fits in the knapsack we take it if not we leave it. Then we look at the next item until we run out of items or we fill the knapsack.

In [16]:
]dinput
greedy←{
 c mw is←⍺
 cw←⍺ weight ⍵
 c ≤ ⍴⍵: ⍺ value ⍵ ⍝ stop if we have looked at all items
 mw=cw: ⍺ value ⍵  ⍝ stop if knapsack is full
 mw<cw: 0          ⍝ stop if knapsack is overfull
 v w←⊃is↓⍨⍴⍵       ⍝ drop items that have already been considered
 ⍺∇⍵,w≤mw-cw       ⍝ recurse with an additional item considered
}

In [29]:
ks4 greedy ⍬

18

## Relaxation/Optimistic solution

For the optimistic solution we sort by value density take items until knapsack is full. And then break the next item into pieces and shove the broken bits into any remaining space.

In [143]:
]dinput
optimistic←{
     c mw is←⍺

     vs ws←↓⍉↑is

     cv ← ⍺ value ⍵
     cw ← ⍺ weight ⍵

     vs ws←↓⍉↑is↓⍨⍴⍵                     ⍝ drop the items already decided
     cw>mw:0                             ⍝ if the weight already exceeds max capacity return 0
     d←(⍴⍵)↓÷⌿⍤1⊢↑is                     ⍝ density
     cv++/d×ws⌊0⌈¯1↓(mw-cw),(mw-cw)-+\ws ⍝ current value plus optimistic guess for remainder

 }

In [144]:
ks4 optimistic ⍬

21.75

## Bound

Given the best solution so far, a problem config, and the list of branches we check each branches and ask what is the optimistic solution. Remember this is not a feasible solution but its better than any possible real solution. If the optimistic solution is not as good as our current best we dont need to explore the branch any further and we discard it. 

In [145]:
]dinput
bound←{
     best ks←⍺
     ⍵⌿⍨best≤ks∘optimistic∘⊃⍤1⊢⍵    ⍝ filter branches where the best solution is less than the optimistic solution 
 }

# Solution
Now that we have all the pieces we just have to put them together and run them a bunch.

In [65]:
⎕←b← ks4 greedy ⍬

18

In [131]:
]dinput
ks_solve←{
     c mw is←⍵
     g←⍵ greedy ⍬
     s←g ⍵∘bound∘branch⍣c⊢⍬
     ⊃(,s)⌷⍨⊂⍒⍵∘value∘⊃⍤1⊢s
}

In [132]:
ks_solve ks4

┌→──────┐
│1 1 0 0│
└~──────┘

## Bigger?

Does it work on a problem we cant solve by hand?

In [133]:
]dinput
read_ks←{
     temp←↓⍎⍤1⊢↑⊃⎕NGET ⍵ 1
     count max_weight←⊃temp
     items←1↓temp
     items←items⌷⍨⊂⍒÷⌿⍤1⊢↑items
     count max_weight items
 }

In [147]:
⎕←ks19←read_ks 'ks_19_0'

┌→─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐ │
│ 19 31181 │ ┌→──────────┐ ┌→──────────┐ ┌→────────┐ ┌→─────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→───────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ │ │
│ │ │16553 40006│ │13504 32708│ │3878 9656│ │4136 10372│ │2945 7390│ │2890 7280│ │1945 4990│ │1865 4830│ │1513 3926│ │1833 4766│ │1022 2744│ │962 2624│ │1107 3114│ │1101 3102│ │1060 3020│ │805 2310│ │689 2078│ │667 2034│ │321 1142│ │ │
│ │ └~──────────┘ └~──────────┘ └~────────┘ └~─────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~───────┘ └~────────┘ └~────────┘ └~────────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ │ │
│ └∊────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘ │
└∊─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [139]:
⎕←ks_solve ks19
⎕←weight∘ks_solve⍨ks19
⎕←value∘ks_solve⍨ks19

┌→────────────────────────────────────┐
│0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0│
└~────────────────────────────────────┘

30996

12248

Is that the best solution? Dunno but it didnt explode my memory and it returned something that looks correct. Seems legit.

## Bigger??

In [148]:
⎕←ks50←read_ks 'ks_50_0'

┌→──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐ │
│ 50 341045 │ ┌→───────────┐ ┌→───────────┐ ┌→───────────┐ ┌→──────────┐ ┌→───────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ ┌→─────────┐ ┌→─────────┐ ┌→─────────┐ ┌→─────────┐ ┌→─────────┐ ┌→─────────┐ ┌→─────────┐ ┌→────────┐ ┌→────────┐ ┌→─────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→────────┐ ┌→───────┐ ┌→────────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→───────┐ ┌→──────┐ ┌→──────┐ ┌→──────┐ ┌→─────┐ │ │
│ │ │96601 231302│ │77174 184848│ │50897 122094│ │29688 71276│ │45136 108372│ │41516 99732│ │23527 56554│ │23552 56804│ │23552 56804│ │13269 32038│ │15572 37644│ │16494 39888│ │14393 34886│ │11326 27552│ │7570 18440│ │7512 18324│ │5516 13532│ │4875 12050│ │4446 10992│ │4436 10972│ │4265 10630│ │3024 7548│ │3383 8466│ │4103 10306│ │2889 7278│ │2721 6942│ │2625 6750│ │1906 4912│ │2181 5662│ │2133 5566│ │2129 5558│ │1795 4690│ │1313 3526│ │1242 3384│ │952 2604│ │1086 3072│ │706 2112│ │492 1484│ │445 1390│ │620 1940│ │419 1338│ │566 1832│ │559 1818│ │348 1196│ │313 1126│ │281 1062│ │246 992│ │217 934│ │103 706│ │67 634│ │ │
│ │ └~───────────┘ └~───────────┘ └~───────────┘ └~──────────┘ └~───────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~──────────┘ └~─────────┘ └~─────────┘ └~─────────┘ └~─────────┘ └~─────────┘ └~─────────┘ └~─────────┘ └~────────┘ └~────────┘ └~─────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~────────┘ └~───────┘ └~────────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~───────┘ └~──────┘ └~──────┘ └~──────┘ └~─────┘ │ │
│ └∊────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘ │
└∊──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [149]:
⎕←ks50_s←ks_solve ks50
⎕←ks50 weight ks50_s
⎕←ks50 value ks50_s

┌→──────────────────────────────────────────────────────────────────────────────────────────────────┐
│1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0│
└~──────────────────────────────────────────────────────────────────────────────────────────────────┘

341012

142156